In [1]:
import torch

/home/adithyay328/Documents/SoftwareProjects/Torch2Vec/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
myModel = torch.load("entire_model.dump")
VOCABSIZE = 3000

In [3]:
import vectorize
vec = vectorize.Vectorizer.load("vectorizer.dump")

In [4]:
# Getting a dict of word : numpy embedding
wordToNumpy = {}

with torch.no_grad():
    for token in vec.dictionary.token2id:
        id = vec.dictionary.token2id[token]

        oneHotDense = torch.zeros( (VOCABSIZE,) )
        oneHotDense[id + 1] = 1
        oneHotSparse = oneHotDense.to_sparse_coo()

        torchEmbedding = myModel(oneHotSparse)
        numpyEmbedding = torchEmbedding.numpy()
        wordToNumpy[token] = numpyEmbedding

/home/adithyay328/Documents/SoftwareProjects/Torch2Vec/venv/lib/python3.8/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [5]:
import pandas as pd
import plotly.express as px

words = [word for word in wordToNumpy]
arrays = [wordToNumpy[word] for word in wordToNumpy]

In [47]:
import sklearn.decomposition

pca = sklearn.decomposition.PCA(50)
pcaVectors = pca.fit_transform(arrays)

In [52]:
import sklearn.manifold

tsne = sklearn.manifold.TSNE(2, perplexity=5, early_exaggeration=30, learning_rate=100)
tsneVecs = tsne.fit_transform(pcaVectors)

/home/adithyay328/Documents/SoftwareProjects/Torch2Vec/venv/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:795: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.



In [53]:
xVals = tsneVecs.T[0]
yVals = tsneVecs.T[1]

In [54]:
df = pd.DataFrame(data={"words" : words, "x" : xVals, "y" : yVals})

In [55]:
fig = px.scatter(df, x="x", y="y", hover_data=["words"])
fig.show()